In [1]:
import json
import os
from pathlib import Path
import time

In [2]:
class Inventory():
    def __init__(self):
        self.records = []
    
    def log(self, res, order):
        fh = open("history.log", "a")
        logline = "\n" + time.ctime() + "| " + res + str(order)
        fh.write(logline)
        fh.close()
    
    def read_records(self):
        fh = open('records.json', "r")
        self.records = json.loads(fh.read())
        fh.close()
    
    def handle_order(self, id, qty):
        itm = None
        order = None
        for item in self.records:
            if item["id"] == id:
                order = item.copy()
                order["instock"] = order["qty"]
                order["qty"] = qty
                itm = item
                break
        if itm:
            if itm["qty"] == 0:
                return ("OUTOFSTOCK", order)
            elif itm["qty"] < order["qty"]:
                return ("FEWINSTOCK", order)
            else:
                return ("INSTOCK", order)
        else:
            return ("NOITEMPRESENT", order)
    
    def update_records(self):
        fh = open('records.json', "w")
        try:
            fh.write(json.dumps(self.records, indent = 4))
            fh.close()
            self.read_records()
            return True
        except:
            return False

    def deduct_inventory(self, order):
        for item in self.records:
            if item["id"] == order["id"]:
                item["qty"] = item["qty"] - order["qty"]
                return True
        return False

    def get_bill(self, order):
        bill = '''
        ***************************************************
        Bill No.\t{}
        Item Id:\t{}
        Name:\t{}
        Price:\t{}

        Requested Quantity:\t{}
        ---------------------------------------------------
        Total Amount:\tRs. {}
        ***************************************************
        '''.format("temp", order["id"], order["name"], order["price"], order["qty"], order["qty"] * order["price"])

        return bill

    def stockUp(self, id):
        self.read_records()
        itm = None
        for item in self.records:
            if item["id"] == id:
                itm = item
                break
        if itm:
            itm["name"] = (input("Enter Item name: ") or itm["name"])
            itm["price"] = int(input("Enter price: ") or itm["price"])
            itm["qty"] = int(input("Enter current stock: ") or itm["qty"])
            self.update_records()
            self.log("ITEM_STOCKED_UP", itm)
            print("\nSuccessfully stocked up")
        else:
            print("\nNo such item exists")
    
    def add_new_item(self, itm):
        self.read_records()
        for item in self.records:
            if item["id"] == itm["id"]:
                return False
        self.records.append(itm)
        self.update_records()
        return True

In [9]:
dashboard = '''
---------------------------------------------------
Inventory Management System

1. Place order
2. stock up inventory
3. History
4. view inventory copy

---------------------------------------------------
'''

inventory = Inventory()
cmd = -1

In [10]:
while cmd != 5:
    print(dashboard)
    cmd = int(input('> ') or -1)

    if cmd == 1:
        inventory.read_records()
        id = input("\nEnter id: ")
        qty = int(input("Enter quantity: "))
        result, order = inventory.handle_order(id, qty)

        if result == "OUTOFSTOCK": 
            print("\nOops, sorry out of stock!!!")
            continue
        elif result == "FEWINSTOCK": 
            print('''
            Requested items: {}
            Items in stock: {}
            '''.format(order["qty"], order["instock"]))
            print("Do you want to buy remaining {} items ? (Y or N): ".format(order["instock"]))
            ans = input()
            if ans == "N":
                print("\nOK order cancelled")
                inventory.log("ORDER_CANCELED", order)
                continue
            elif ans == "Y":
                order["qty"] = order["instock"]
        elif result == "NOITEMPRESENT": 
            print("\nNo Item Exists")
            continue

        res = inventory.deduct_inventory(order)
        res &= inventory.update_records()
        inventory.log("ITEM_DEDUCTED", order)
        if res:
            print(inventory.get_bill(order))
            print("\nOrder successfull")
        else:
            print("\nERROR: Something occured while dedcuting inventory")
            
    elif cmd == 2:
        c = int(input("\n1. STock up existing item.\n2. Add new Item.\n\n> "))
        if c == 1:
            id = input("\nEnter id: ")
            inventory.stockUp(id)
        elif c == 2:
            itm = {}
            itm["id"] = input("\nEnter id: ")
            itm["name"] = input("Enter Item name: ")
            itm["price"] = int(input("Enter price: ") or 100)
            itm["qty"] = int(input("Enter current stock: ") or "0")
            if itm["id"] and itm["name"]:
                inventory.add_new_item(itm)
                print("\nAdded new item")
                inventory.log("NEW_ITEM_ADDED", itm)
            else:
                print("\nPlease enter valid details")
    elif cmd == 3:
        os.startfile("history.log")
    elif cmd == 4:
        home = Path.home()
        name = (time.ctime() + ".json").replace(":", "-")
        path = os.path.join(home, "Documents", "InventorySystem")
        if not os.path.exists(path):
            os.makedirs(path)
        fh = open(path+"\\"+name, "w")
        fh.write(open("records.json", "r").read())
        fh.close()
        os.startfile(path+"\\"+name)
        print("\nSaved inventory copy" + path+"\\"+name)
        inventory.log("INVENTORY_COPY_CREATED", {})
    else:
        pass

print('\nExiting... \n')


---------------------------------------------------
Inventory Management System

1. Place order
2. stock up inventory
3. History
4. view inventory copy

---------------------------------------------------

> 1

Enter id: 50275
Enter quantity: 10

Oops, sorry out of stock!!!

---------------------------------------------------
Inventory Management System

1. Place order
2. stock up inventory
3. History
4. view inventory copy

---------------------------------------------------

> 1

Enter id: 50276
Enter quantity: 10

        ***************************************************
        Bill No.	temp
        Item Id:	50276
        Name:	Apple macbook air
        Price:	2860

        Requested Quantity:	10
        ---------------------------------------------------
        Total Amount:	Rs. 28600
        ***************************************************
        

Order successfull

---------------------------------------------------
Inventory Management System

1. Place order
2. stock u